In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle
from parse_xml_annotations import *
from reinforcement import *
from metrics import *
from features import *
from PIL import Image
import random
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

Using TensorFlow backend.


In [2]:
with open('data/pid_map_image.txt', 'rb') as f:
    u = pickle._Unpickler(f)
    u.encoding = 'latin1'
    data = u.load()
# for k, _ in enumerate(roidb):
#     roidb[k]['image'] =  os.path.join(os.getcwd(), '/'.join(roidb[k]['image'].split('/')[5:]))

In [3]:
scale_subregion = float(3)/4
scale_mask = float(1)/(scale_subregion*4)
replay = []
h = 0
buffer_experience_replay = 1000
reward=0

In [4]:
from __future__ import print_function
#import model

class ModelController(object):

    def __init__(self, data, pretrained_model=None, batch_size=32):
        """Initialize the SolverWrapper."""
        self.data = data
        self.num_data = len(self.data)
        self.batch_size = batch_size
        self.pretrained_model=pretrained_model
        
        #tf.reset_default_graph()
        #self.images_placeholder = tf.placeholder(tf.float32, shape=(1,224,224,3))
        #self.my_feature_map_extractor_model = model.get_feature_map_extractor_model(self.images_placeholder, None, None, is_training=False)
        
        #self.sess = tf.Session(config=tf.ConfigProto(allow_soft_placement=True))
        
        self.rl_model = build_model()
        
    def load_feature_map_model(self, checkpoint_path):
        path_vgg = "./vgg16_weights.h5"
        self.feature_map_extractor_model = obtain_compiled_vgg_16(path_vgg)
        
    def get_next_minibatch(self):
        l = []
        inds = np.random.choice(range(self.num_data), self.batch_size, replace=False)
        for i in inds:
            node = np.random.choice(range(len(self.data[i])), 2)
            l.append([
                [i, self.data[i][node[0]]],
                [i, self.data[i][node[1]]]
            ])
        return l
    
    def save_model(self):
        print('\nsave model')
        string = './models/model' + str(t) + '_epoch_' + str(i) + '.h5'
        string2 = './models/model' + str(t) + '.h5'
        self.rl_model.save_weights(string, overwrite=True)
        self.rl_model.save_weights(string2, overwrite=True)

    def train_model(self, max_iters=10, number_of_steps=10):
        for i in range(max_iters):
            print(i)
            nodes = self.get_next_minibatch()
            for node in nodes:
                pid = node[0][0]
                target_data = node[0][1]
                search_data = node[1][1]

                masked = 0
                not_finished = 1

                target_image = np.array(Image.open(target_data['image']))
                bbox = target_data['boxes'][np.where(target_data['gt_pids']==pid)[0][0]]
                target_image = target_image[bbox[1]:bbox[3],bbox[0]:bbox[2]]

                search_image = np.array(Image.open(search_data['image']))
                annotation = search_data['boxes'][np.where(search_data['gt_pids']==pid)[0][0]].reshape(1,-1).astype(np.int32)
                gt_masks = generate_bounding_box_from_annotation(annotation, search_image.shape)
                region_mask = np.ones([search_image.shape[0], search_image.shape[1]])
                shape_gt_masks = np.shape(gt_masks)
                array_classes_gt_objects = np.array([pid])
                available_objects = np.ones(np.size(array_classes_gt_objects))

                gt_mask = gt_masks[:, :, 0]
                step = 0
                new_iou = 0
                last_matrix = np.zeros([1])
                region_image = search_image
                offset = (0, 0)
                size_mask = (search_image.shape[0], search_image.shape[1])
                original_shape = size_mask
                old_region_mask = region_mask
                region_mask = np.ones([search_image.shape[0], search_image.shape[1]])

                iou, new_iou, last_matrix, index = follow_iou(gt_masks, region_mask, array_classes_gt_objects,
                                pid, last_matrix, available_objects)
                new_iou = iou
                gt_mask = gt_masks[:, :, index]
                # init of the history vector that indicates past actions (6 actions * 4 steps in the memory)
                history_vector = np.zeros([24])
                # computation of the initial state
                search_iv = get_image_vector(region_image, self.feature_map_extractor_model)
                target_iv = get_image_vector(target_image, self.feature_map_extractor_model)
                state = get_state(target_iv, search_iv, history_vector)
                print(state.shape)
                print(state)
                # status indicates whether the agent is still alive and has not triggered the terminal action
                status = 1
                action = 0
                reward = 0
                if step > number_of_steps:
                    #background = draw_sequences(i, k, step, action, draw, region_image, background,
                                                #path_testing_folder, iou, reward, gt_mask, region_mask, image_name,
                                                #bool_draw)
                    step += 1

                while (status == 1) & (step < number_of_steps) & not_finished:
                   # category = int(array_classes_gt_objects[0]-1)
                    qval = self.rl_model.predict(state.T, batch_size=1)
    #                     background = draw_sequences(i, k, step, action, draw, region_image, background,
    #                                                 path_testing_folder, iou, reward, gt_mask, region_mask, image_name,
    #                                                 bool_draw)
                    step += 1
                    # we force terminal action in case actual IoU is higher than 0.5, to train faster the agent
                    if (i < 100) & (new_iou > 0.5):
                        action = 6
                    # epsilon-greedy policy
                    elif random.random() < 0.1:
                        action = np.random.randint(1, 7)
                    else:
                        action = (np.argmax(qval))+1

                    # terminal action
                    if action == 6:
                        iou, new_iou, last_matrix, index = follow_iou(gt_masks, region_mask,
                                                                      array_classes_gt_objects, pid,
                                                                      last_matrix, available_objects)
                        gt_mask = gt_masks[:, :, index]
                        reward = get_reward_trigger(new_iou)
#                         background = draw_sequences(i, k, step, action, draw, region_image, background,
#                                                     path_testing_folder, iou, reward, gt_mask, region_mask,
#                                                     image_name, bool_draw)
                        step += 1
                    # movement action, we perform the crop of the corresponding subregion
                    else:
                        region_mask = np.zeros(original_shape)
                        size_mask = (size_mask[0] * scale_subregion, size_mask[1] * scale_subregion)
                        if action == 1:
                            offset_aux = (0, 0)
                        elif action == 2:
                            offset_aux = (0, size_mask[1] * scale_mask)
                            offset = (offset[0], offset[1] + size_mask[1] * scale_mask)
                        elif action == 3:
                            offset_aux = (size_mask[0] * scale_mask, 0)
                            offset = (offset[0] + size_mask[0] * scale_mask, offset[1])
                        elif action == 4:
                            offset_aux = (size_mask[0] * scale_mask, 
                                          size_mask[1] * scale_mask)
                            offset = (offset[0] + size_mask[0] * scale_mask,
                                      offset[1] + size_mask[1] * scale_mask)
                        elif action == 5:
                            offset_aux = (size_mask[0] * scale_mask / 2,
                                          size_mask[0] * scale_mask / 2)
                            offset = (offset[0] + size_mask[0] * scale_mask / 2,
                                      offset[1] + size_mask[0] * scale_mask / 2)
                        region_image = region_image[int(offset_aux[0]):int(offset_aux[0]) + int(size_mask[0]),
                                       int(offset_aux[1]):int(offset_aux[1]) + int(size_mask[1])]
                        region_mask[int(offset[0]):int(offset[0]) + int(size_mask[0]), int(offset[1]):int(offset[1]) + int(size_mask[1])] = 1
                        iou, new_iou, last_matrix, index = follow_iou(gt_masks, region_mask,
                                                                      array_classes_gt_objects, pid,
                                                                      last_matrix, available_objects)
                        gt_mask = gt_masks[:, :, index]
                        reward = get_reward_movement(iou, new_iou)
                        iou = new_iou
                    history_vector = update_history_vector(history_vector, action)
                    #new_state = get_state(region_image, history_vector, self.feature_map_extractor_model)
                    search_iv = get_image_vector(region_image, self.feature_map_extractor_model)
                    new_state = get_state(target_iv, search_iv, history_vector)
                    # Experience replay storage
                    if len(replay) < buffer_experience_replay:
                        replay.append((state, action, reward, new_state))
                    else:
                        if h < (buffer_experience_replay-1):
                            h += 1
                        else:
                            h = 0
                        h_aux = h
                        h_aux = int(h_aux)
                        replay[h_aux] = (state, action, reward, new_state)
                        minibatch = random.sample(replay, batch_size)
                        X_train = []
                        y_train = []
                        # we pick from the replay memory a sampled minibatch and generate the training samples
                        for memory in minibatch:
                            old_state, action, reward, new_state = memory
                            old_qval = model.predict(old_state.T, batch_size=1)
                            newQ = model.predict(new_state.T, batch_size=1)
                            maxQ = np.max(newQ)
                            y = np.zeros([1, 6])
                            y = old_qval
                            y = y.T
                            if action != 6: #non-terminal state
                                update = (reward + (gamma * maxQ))
                            else: #terminal state
                                update = reward
                            y[action-1] = update #target output
                            X_train.append(old_state)
                            y_train.append(y)
                        X_train = np.array(X_train)
                        y_train = np.array(y_train)
                        X_train = X_train.astype("float32")
                        y_train = y_train.astype("float32")
                        X_train = X_train[:, :, 0]
                        y_train = y_train[:, :, 0]
                        hist = model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=1, verbose=0)
                        models[0][category] = model
                        state = new_state
                    if action == 6:
                        status = 0
                        masked = 1
                        sys.stdout.write('\r'+str(j)+'/'+str(np.size(image_names))+'  step:'+str(step))
                        # we mask object found with ground-truth so that agent learns faster
                        image = mask_image_with_mean_background(gt_mask, image)
                    else:
                        masked = 0
                available_objects[index] = 0
        if (i+1)%1000 is 0:
            self.save_model()
            
ct = ModelController(data, pretrained_model=None, batch_size=1)
ct.load_feature_map_model('./resnet_v1_50.ckpt')
ct.get_next_minibatch()
ct.train_model()

0
(10776, 1)
[[ 0.18628812]
 [ 0.32921845]
 [ 0.31390208]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
1
(10776, 1)
[[ 0.27941975]
 [ 0.40466195]
 [ 0.26671895]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
2
(10776, 1)
[[ 0.23606631]
 [ 0.40666956]
 [ 0.2670961 ]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
3
(10776, 1)
[[ 0.17810456]
 [ 0.50084829]
 [ 0.33183503]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
4
(10776, 1)
[[ 0.15912551]
 [ 0.26796705]
 [ 0.20006512]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
5
(10776, 1)
[[ 0.18522516]
 [ 0.2112546 ]
 [ 0.19421135]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
6
(10776, 1)
[[ 0.18962294]
 [ 0.36071432]
 [ 0.26621214]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
7
(10776, 1)
[[ 0.29220316]
 [ 0.33899677]
 [ 0.42536509]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
8
(10776, 1)
[[ 0.18579939]
 [ 0.38536283]
 [ 0.30417132]
 ..., 
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]]
9